In [1]:

import mysql.connector  
import configparser
import json
from deepface import DeepFace
from deepface.commons import package_utils, folder_utils
import os

from datetime import datetime
import hashlib

import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
import math
import time

#!pip install deepface
from deepface import DeepFace
from deepface.commons import package_utils, folder_utils

In [2]:
model = DeepFace.build_model("Facenet")

In [3]:
facial_img_paths = []
#Available at: https://github.com/serengil/deepface/tree/master/tests/dataset
for root, directory, files in os.walk("../dataset"):
    for file in files:
        if '.jpg' in file:
            facial_img_paths.append(root+"/"+file)

In [4]:
instances = []
 
for i in tqdm(range(0, len(facial_img_paths))):
#for i in tqdm(range(0, 10)):
    facial_img_path = facial_img_paths[i]   
    embedding = DeepFace.represent(img_path = facial_img_path, model_name = "Facenet", enforce_detection=False)[0]["embedding"]
    objs = DeepFace.analyze(img_path = facial_img_path) 
    instance = []
    instance.append(facial_img_path)
    instance.append(embedding)
    instance.append(objs[0]["dominant_emotion"])
    instance.append(objs[0]["age"])
    instance.append(objs[0]["dominant_gender"])
    instance.append(objs[0]["dominant_race"])
    instances.append(instance)

100%|██████████| 57/57 [00:43<00:00,  1.31it/s]


In [5]:
df = pd.DataFrame(instances, columns = ["img_name", "embedding","dominant_emotion", "age", "dominant_gender", "dominant_race"])
df.head()

,img_name,embedding,dominant_emotion,age,dominant_gender,dominant_race
0,../dataset/img22.jpg,"[1.8546762466430664, 0.4593982398509979, -1.19...",happy,32,Woman,white
1,../dataset/img36.jpg,"[1.0648695230484009, 0.014331400394439697, -0....",neutral,32,Man,latino hispanic
2,../dataset/img37.jpg,"[1.67017662525177, 1.3304144144058228, -0.9308...",neutral,28,Man,latino hispanic
3,../dataset/img23.jpg,"[1.7130699157714844, 2.1187288761138916, -1.11...",happy,33,Woman,white
4,../dataset/img35.jpg,"[0.6689144372940063, 0.9128508567810059, -0.13...",neutral,25,Man,white


In [6]:

# Assuming your DataFrame is named df
df['img_name'] = df['img_name'].str.replace('../dataset/', '')

# Display the modified DataFrame
df.head()


,img_name,embedding,dominant_emotion,age,dominant_gender,dominant_race
0,img22.jpg,"[1.8546762466430664, 0.4593982398509979, -1.19...",happy,32,Woman,white
1,img36.jpg,"[1.0648695230484009, 0.014331400394439697, -0....",neutral,32,Man,latino hispanic
2,img37.jpg,"[1.67017662525177, 1.3304144144058228, -0.9308...",neutral,28,Man,latino hispanic
3,img23.jpg,"[1.7130699157714844, 2.1187288761138916, -1.11...",happy,33,Woman,white
4,img35.jpg,"[0.6689144372940063, 0.9128508567810059, -0.13...",neutral,25,Man,white


In [7]:
def read_config(database_name):
    config = configparser.ConfigParser()
    config.read('../config.ini')
    #config.read('config.ini')
    return config[database_name]

In [8]:
config_data = read_config('database')

In [9]:
host = config_data['server']
database = config_data['database']
username = config_data['username']
password = config_data['password']

In [10]:
host

'localhost'

In [11]:
db_connection = mysql.connector.connect(
    host=host,
    user=username,
    password=password,
    database=database
)

cursor = db_connection.cursor()

for index, instance in tqdm(df.iterrows(), total=df.shape[0]):
    img_name = instance["img_name"]
    embeddings = instance["embedding"]
    age = instance["age"]
    dominant_emotion = instance["dominant_emotion"]
    dominant_gender = instance["dominant_gender"]
    dominant_race = instance["dominant_race"]
     
    # Convert embeddings to JSON string
    embeddings_json = json.dumps(embeddings)
    
    # Insert into face_meta table
    current_time = datetime.now()
    unique_key = hashlib.sha256(str(current_time).encode()).hexdigest()

    # Insert into face_meta table
    #insert_statement = "INSERT INTO face_meta (ID, IMG_NAME, EMBEDDING, AGE, DOMINANT_EMOTION, DOMINANT_GENDER, DOMINANT_RACE) VALUES (?, ?, ?, ?, ?, ?, ?)"
    insert_statement = "INSERT INTO face_meta (ID, IMG_NAME, EMBEDDING, AGE, DOMINANT_EMOTION, DOMINANT_GENDER, DOMINANT_RACE) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    insert_args = (unique_key, img_name, embeddings_json, age, dominant_emotion, dominant_gender, dominant_race)
    #print(insert_args)
    cursor.execute(insert_statement, insert_args)
   
    # Insert into face_embeddings table
    for i, embedding in enumerate(embeddings):
        insert_statement = "INSERT INTO face_embeddings (FACE_ID, DIMENSION, VALUE) VALUES (%s, %s, %s)"
        insert_args = (unique_key, i, embedding)
        cursor.execute(insert_statement, insert_args)



100%|██████████| 57/57 [00:00<00:00, 149.40it/s]


In [37]:
db_connection.commit()

In [38]:
cursor.close()
db_connection.close()